In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math, copy
import re

import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix, hstack


In [ ]:
#import sys
#print(sys.executable)

In [ ]:
def load_data(file_path, sample_size=10000 ):
    df = pd.read_csv(file_path, encoding="latin1")

    df =  df.sample(n=min(sample_size, len(df)), random_state=42)

    df = df.loc[df["price"] > 0]


    df[["main_category", "sub_category", "sub_sub_category"]] = df["category_name"].str.split("/", expand=True, n=2)
    df["name_length"] = df.item_description.str.len()
    df["desc_length"] = df.item_description.str.len()
    #data[["main_category", "sub_category", "sub_sub_category"]] = data["category_name"].str.split("/", expand=True, n=2)

    df.brand_name.fillna("Unknown", inplace=True)
    df.main_category.fillna("Unknown", inplace=True)
    df.sub_category.fillna("Unknown", inplace=True)
    df.sub_sub_category.fillna("Unknown", inplace=True)
    
    return df

load_data("/Users/user/Documents/Projects/ml_projects/ecommerce_ml_system/data/raw/train2.csv", sample_size=10000)



In [ ]:
def feature_engineering(df):

    y = np.log1p(df['price'])

    x = pd.DataFrame()

    x["item_condition_id"] = df["item_condition_id"]
    #x["brand_name"] = df["brand_name"]
    x["shipping"] = df["shipping"]
    x["item_description"] = df["item_description"]
    x["main_category"] = df["main_category"]
    x["sub_category"] = df["sub_category"]
    x["sub_sub_category"] = df["sub_sub_category"]
    x["name"] = df["name"]
    x["name_length"] = df["name_length"]
    x["desc_length"] = df["desc_length"]

    encoder = OneHotEncoder(sparse_output=True) 
    categorical_features = ["main_category", "sub_category", "sub_sub_category"]

    # Fit and transform
    encoded_category = encoder.fit_transform(x[categorical_features])
    #df_encoded_category = pd.DataFrame(encoded_category.toarray(), columns=encoder.get_feature_names_out(categorical_features))

    x = x.drop(columns=categorical_features)
    #X_features = pd.concat([x.reset_index(drop=True), df_encoded_category.reset_index(drop=True)], axis=1)

    x["name"] = x["item_description"].str.lower().str.strip()
    x["item_description"] = x["item_description"].str.lower().str.strip()
    x["item_description"] =  x["item_description"].replace("No description yet", "")

    tfidf_name = TfidfVectorizer(max_features=5000, ngram_range=(1, 2), stop_words="english")
    tfidf_desc = TfidfVectorizer(max_features=10000, ngram_range=(1, 2), stop_words="english")

    X_name_tfidf = tfidf_name.fit_transform(x["name"])
    X_desc_tfidf = tfidf_desc.fit_transform(x["item_description"])

    x = x.drop(columns=['name', 'item_description'])

    # Convert other features to array
    X_numeric = csr_matrix(x.values)
    
     # Combine everything
    X_final = hstack([X_numeric, encoded_category, X_name_tfidf, X_desc_tfidf])

    return X_final, y
X, y =feature_engineering(load_data("/Users/user/Documents/Projects/ml_projects/ecommerce_ml_system/data/raw/train2.csv", sample_size=10000))


In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
type(X)

In [ ]:
type(y)

In [2]:
data = pd.read_csv("/Users/user/Documents/Projects/ml_projects/ecommerce_ml_system/data/raw/train2.csv", encoding="latin1")

In [3]:
data = data.copy()

In [4]:
data["brand_name"].fillna("Unknown", inplace=True)

In [ ]:
#According to information available on Mercari Website

# 1 stands for new
# 2 stands for fairly new
# 3 stands for Good
# 4 stands for Bad
# 5 stands for Very Poor

In [5]:
brand_count = data["brand_name"].value_counts()
data["brand_freq"] = data["brand_name"].map(brand_count)

In [6]:
brand_avg_price = data.groupby("brand_name")["price"].mean()
data["brand_target"] = data["brand_name"].map(brand_avg_price)

In [7]:
top_brands = data["brand_name"].value_counts().head(100).index
data["brand_grouped"] = data["brand_name"].where(data["brand_name"].isin(top_brands), "Other")

In [9]:
data.columns

Index(['train_id', 'name', 'item_condition_id', 'category_name', 'brand_name',
       'price', 'shipping', 'item_description', 'brand_freq', 'brand_target',
       'brand_grouped'],
      dtype='object')

In [8]:
data.head(20)

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,brand_freq,brand_target,brand_grouped
0,0,Hero 77 fountain pen,2,Other/Office supplies/Writing,Unknown,12.0,1,"For sale a brand new Hero 77 fountain pen, doe...",62991,21.229803,Unknown
1,1,14K Yellow Gold Earrings,3,Women/Jewelry/Earrings,Unknown,20.0,0,14k black Onyx earrings Good condition Final sale,62991,21.229803,Unknown
2,2,New balance 2-in 1 size S dry fit shorts,2,Women/Athletic Apparel/Shorts,New Balance,10.0,0,"Brand new never worn, but I tore the tag off w...",83,28.240964,Other
3,3,Zella black workout tank w mesh cut out,3,Women/Athletic Apparel/Shirts & Tops,Zella,15.0,1,Zella black workout tank with mesh cut outs.,14,17.571429,Other
4,4,NWT Lilly Pulitzer gabby dress sz 8,1,"Women/Dresses/Above Knee, Mini",Lilly Pulitzer,75.0,0,New with tags!! Size 8.,288,40.461806,Lilly Pulitzer
5,5,FREESHIP summer navyblue floral sundress,2,"Women/Dresses/Above Knee, Mini",FOREVER 21,13.0,1,-size small no padding (wear a padded bra if u...,1533,12.923679,FOREVER 21
6,6,Brand New With Tag American Eagle Romper,1,Women/Dresses/Other,American Eagle,20.0,1,"Brand new, originally spent [rm]. Dark charcoa...",1367,15.781273,American Eagle
7,7,Little Debbie Barbie,1,Kids/Toys/Dolls & Accessories,Barbie,18.0,0,NWT anniversary edition Barbie,119,22.352941,Other
8,8,4th of July Slime,1,Kids/Toys/Hobbies,Elmers,10.0,0,4 th of July Slime Floam Bundle Set of 3 2.5 o...,143,8.174825,Other
9,9,Skechers Size 8.5,3,Women/Shoes/Athletic,SKECHERS,13.0,0,"Women's Skechers size 8.5 white, grey, and lig...",83,20.542169,Other


In [ ]:
data["name_counts"] = data["name"].str.split().str.len()

In [ ]:
data["brand_grouped"].nunique()

In [ ]:
data.isna().sum()

In [ ]:
plt.figure(figsize=(15,5))
plt.title("Price Distribution")
sns.boxplot(data.price, showfliers=False)
plt.xlabel("Price", fontsize=15)

plt.show()

In [ ]:
plt.subplot(1, 2, 1)
(data['price']).plot.hist(bins=50, figsize=(12, 6), edgecolor = 'white', range = [0, 350])
plt.xlabel('price', fontsize=12)
plt.title('Price Distribution', fontsize=12)

plt.subplot(1, 2, 2)
np.log1p(data['price']).plot.hist(bins=50, figsize=(12, 6), edgecolor='white')
plt.xlabel('log(price)+1', fontsize=12)
plt.title('Price Distribution', fontsize=12)

In [ ]:
#% of zero or very low prices

In [ ]:
data.price.value_counts(normalize=True)

In [ ]:
(data["price"] == 0).mean()*100

In [ ]:
(data.price <= 1).mean()*100

In [ ]:
(data.price <= 2).mean()*100

In [ ]:
(data.price <= 3).mean()*100

In [ ]:
(data.price >= 100).mean()*100

In [ ]:
(data.price <= 1).sum()

In [ ]:
#Outliers (very expensive items)


In [ ]:
price_cutoff = 100

In [ ]:
expensive_df = data[data["price"] >= price_cutoff]
expensive_df

In [ ]:
(data.price >= 100).mean()*100

In [ ]:
# Even though it’s only 3%, price data is right-skewed.

#That means:
# Those 2% values are numerically large. They contribute disproportionately to:
# RMSE
# MSE
# Gradient updates
#So:
#Count ≠ impact
# A few large prices can still dominate training if you model raw price.
# So: I will keep the 3% skewed data but will use log(price) as target rather than raw price:
# Because;
# Log transform compresses those large values
# 3% expensive items become manageable
# They won’t skew training
# It keep valuable signal (luxury categories, brands)


#Conclusion
# Only 3% of items are priced ≥ $100. These values are rare but expected for certain categories and brands. 
# Since price is highly right-skewed, a log-transformation of the target will be applied to reduce the influence 
# of extreme values while preserving legitimate high-price signals.”

In [ ]:
# Checking the Brands and Category with prices >= 100
data.loc[data.price >= 100, ["category_name", "brand_name"]].head(10)

In [ ]:
# Conclusion
# * Price is not normally distributed

# * Most prices are clustered at the low end

# * A small number of items are extremely expensive (About 3%)

# * Zero / near-zero prices exist and need handling

# * Log-transforming price is justified

In [ ]:
# 2. Price vs Shipping (shipping)
# Question: Do items with seller-paid shipping cost more?

In [ ]:
data.shipping.value_counts(normalize=True)

# 55% of items shipped were paid by buyers
# 45% of items shipped were paid by sellers

In [ ]:
sellers_paid = data.loc[data["shipping"] == 1, "price"]
buyers_paid = data.loc[data["shipping"] == 0, "price"]

price_log = np.log1p(data.price)
sellers_paid_log_price = price_log[data["shipping"] == 1]
buyers_paid_log_price = price_log[data["shipping"] == 0]

In [ ]:
#Using median because using mean is dangerous because Price is * right-skewed * has outliers * has a long expensive tail
# Using median * represents the “typical” item * is robust to outliers * reflects real buyer behavior
# So when you ask: “Do seller-paid shipping items cost more?”
# Median answers: “What does a typical seller-paid item cost?”
sellers_paid.median()

#Average price of item when sellers paid for shipping is $14

In [ ]:
buyers_paid.median()

#Average price of item when buyers paid for shipping is $20

In [ ]:
#Mean is acceptable when: You already log-transformed price
sellers_paid_log_price.mean()

In [ ]:
buyers_paid_log_price.mean()

In [ ]:
# Conclusion is Seller-paid shipping items have a higher median price 
# than buyer-paid shipping items, suggesting shipping cost is partially embedded in the listed price.

In [ ]:
# 3. Price vs Item Condition
# Question: Does condition monotonically affect price?

#According to information available on Mercari Website

# 1 stands for new
# 2 stands for fairly new
# 3 stands for Good
# 4 stands for Bad
# 5 stands for Very Poor

In [ ]:
new = data.loc[data["item_condition_id"] == 1, "price"]
fairly_new = data.loc[data["item_condition_id"] == 2, "price"]
good = data.loc[data["item_condition_id"] == 3, "price"]
bad = data.loc[data["item_condition_id"] == 4, "price"]
very_poor = data.loc[data["item_condition_id"] == 5, "price"]

In [ ]:
data.groupby("item_condition_id")["price"].median().sort_index()

In [ ]:
#Conclusion
# Condition 5 (worst) median price (19) > Condition 1 (best) (18)
#On inspection: * Brands = high-end / luxury * Categories = electronics / collectibles
# This explains the anomaly.
#Even though condition is normally ordinal, other features (brand & category) override it.
# Key EDA insight
# Item condition is generally predictive → median decreases as condition worsens in general
# But outliers exist due to: * High-value brands * High-value categories
#This is market reality, not an error.

In [ ]:
iqr = data.groupby("item_condition_id")["price"].quantile(0.75) - data.groupby("item_condition_id")["price"].quantile(0.25)
iqr

In [ ]:
data.groupby("item_condition_id")["price"].quantile(0.99)

In [ ]:
# Are expensive items in bad condition realistic? Are they tied to luxury brands or electronics?

outliers = data[data["price"] > data.groupby("item_condition_id")["price"].transform("quantile", 0.99)]
outliers[["item_condition_id", "price", "category_name", "brand_name"]].head(10)

In [ ]:
# Median price decreases as item condition worsens, confirming condition as a meaningful ordinal feature. 
# While price distributions overlap across conditions, higher-condition items show consistently higher medians and wider spreads.

In [ ]:
# 4. Price vs Category

In [ ]:
data.category_name

In [ ]:
data.category_name.value_counts()[:15]

In [ ]:
data[["main_category", "sub_category", "sub_sub_category"]] = data["category_name"].str.split("/", expand=True, n=2)

In [ ]:
# Checking price per category

data.groupby("main_category")["price"].median()

In [ ]:
data.groupby("sub_category")["price"].median()

In [ ]:
data.groupby("sub_sub_category")["price"].median()

In [ ]:
# Category Frequency

data["main_category"].value_counts()

In [ ]:
data["sub_category"].value_counts()

In [ ]:
data["sub_sub_category"].value_counts()

In [ ]:
percentile_95 = data.price.quantile(0.95)  # Top 5%
percentile_99 = data.price.quantile(0.99)  # Top 1%

In [ ]:
# Filter expensive items

top_5 = data[data["price"] >= percentile_95]
top_1 = data[data["price"] >= percentile_99]

In [ ]:
# Which catgeory dominate the tails

top_5["main_category"].value_counts().head(10)

In [ ]:
top_1["main_category"].value_counts().head(10)

In [ ]:
top_5["sub_category"].value_counts().head(10)

In [ ]:
top_1["sub_category"].value_counts().head(10)

In [ ]:
# Compare against the full dataset. This tells you whether a category is over-represented.

top_1["main_category"].value_counts(normalize=True).head(10)

In [ ]:
data["main_category"].value_counts(normalize=True).head(10)

In [ ]:
# median price inside the tail. This tells you whether prices are consistently high or just spiky.

top_1.groupby("main_category")["price"].median().sort_values(ascending=False)

In [ ]:
# Inspect few rows to make sure the top 1% brands, categories are high end

top_1[["price", "main_category", "sub_category", "brand_name"]].head(10)

In [ ]:
q = data.groupby("main_category")["price"].quantile([0.25, 0.75]).unstack()
q["IQR"] = q[0.75] - q[0.25]
q.sort_values("IQR", ascending=False)


In [ ]:
# 5. Price vs Brand

# Check median for brands that appear frequently

brand_counts = data.brand_name.value_counts()
frequent_brands = brand_counts[brand_counts >= 500].index

median_frequent_brands =(data[data["brand_name"].isin(frequent_brands)].groupby("brand_name")["price"].median().sort_values(ascending=False))
median_frequent_brands

In [ ]:
# Brands in the expensive tail (most important). Only 2% of items are above $100. So, check brands that are in this expensive tail

expensive_items = data[data["price"] >= 100]
expensive_brand_counts = expensive_items["brand_name"].value_counts()
expensive_brand_counts.head(20)

In [ ]:
# Compare against overall brand frequency (critical step)
# Raw counts are not enough.
# You must compare: * % of a brand in expensive tail vs * % of that brand in entire dataset

expensive_brand_share = expensive_items["brand_name"].value_counts(normalize=True)
overall_brand_share = data["brand_name"].value_counts(normalize=True)

brand_tail_ratio = (expensive_brand_share / overall_brand_share).dropna()
brand_tail_ratio.sort_values(ascending=False).head(20)

In [ ]:
# 5. Price vs Text Length

In [ ]:
# Create a function to count the lenght of each description

def length(description):
    count = 0
    for len in description.split():
        count += 1
    return count

In [ ]:
desc_len_series = []
for i in data["item_description"]:
    temp = []
    temp.append(i)
    temp.append(length(str(i)))
    desc_len_series.append(temp)
desc_len_series[0:3]

In [ ]:
name_len_series = []
for i in data["name"]:
    temp = []
    temp.append(i)
    temp.append(length(str(i)))
    name_len_series.append(temp)

In [ ]:
desc_df = pd.DataFrame(desc_len_series, columns=["description", "desc_length"])
desc_df.head(2)

In [ ]:
name_df = pd.DataFrame(name_len_series, columns=["name", "name_length"])
name_df.head(2)

In [ ]:
data["desc_length"] = desc_df["desc_length"]
data["name_length"] = name_df["name_length"]

In [ ]:
data.head(4)

In [ ]:
plt.figure(figsize=(10,5))
sns.scatterplot(x=data.desc_length, y=data.desc_length.value_counts())
plt.title('Scatter-plot of description length',fontsize=15)
plt.xlabel("Description Length in words",fontsize=10)
plt.ylabel("Frequency",fontsize=10)

In [ ]:
desc_length_count = data["desc_length"].value_counts()

plt.figure(figsize=(15, 5))
sns.barplot(desc_length_count.index[0:11])
plt.title('Item Description with most frequent number of words',fontsize=15)
plt.xticks(rotation = 0,wrap = True,fontsize = 10)
plt.xlabel('Number of words',fontsize=10)
plt.ylabel('Frequency',fontsize=10)
plt.show()

In [ ]:
# NLP

In [ ]:
# Lower case all

data["item_description"] = data["item_description"].str.lower()
data["name"] = data["name"].str.lower()

# Remove white and trailing spaces

data["item_description"] = data["item_description"].str.strip()
data["name"] = data["name"].str.strip()

In [ ]:
nltk.download(['wordnet', 'stopwords', 'punkt', 'omw-1.4', 'punkt_tab'])

In [ ]:
# Word cloud for item description

from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from PIL import Image

# Checking the length of stopwords
len(stop_words)

# Adding Punctuation to stopwords
from string import punctuation
punctuation = list(punctuation)
stop_words.update(punctuation)

In [ ]:
doc1 = []
for row in range(0, data.shape[0]):
    text = str(data["item_description"][row])
    text = re.sub("[^a-zA-Z]", " ", text) # remove non-alphabetic chars
    text = nltk.word_tokenize(text.lower()) # tokenize & lowercase
    text = [lemmatizer.lemmatize(word) for word in text if word not in stop_words and len(word) > 2]
    text = " ".join(text)
    doc1.append(text)

# join string
doc2 = "".join(doc1)

In [ ]:
# wordcloud visualization
#img = np.array(Image.open("../input/images/proj images/cmt.png")) 
#wordcloud = WordCloud(
 #   width=1000,
#    height=500,
 #   background_color="white",
#    stopwords=stop_words,
#    max_words=100
#).generate(doc2)

# Plot
#plt.figure(figsize=(15, 7))
#plt.imshow(wordcloud, interpolation="bilinear")
#plt.axis("off")
#plt.show()

In [ ]:
data.isna().sum()

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data.isna().sum()

In [ ]:
data.groupby("category_name").count()

In [ ]:
data.head()

In [ ]:
data.isna().sum()

In [ ]:
#data["brand_name"] = data["brand_name"].fillna("Unknown")
#data["main_category"] = data["main_category"].fillna("Other")
#data["sub_category"] = data["sub_category"].fillna("Other")
#data["sub_sub_category"] = data["sub_sub_category"].fillna("Other")

In [ ]:
data.head()

In [ ]:
data[data["main_category"] == "Other"]

In [ ]:
data.isna().sum()